In [3]:
# Create json configs
import pandas as pd
import json
import numpy as np

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


date_range = pd.date_range('2021-10-01', '2021-10-31', freq='24h')
# date_range = pd.date_range('2021-12-15', '2021-12-15', freq='24h')
date_range = [dr.strftime('%Y-%m-%d') for dr in date_range]

configs_dir = 'results/test_data/configs/'
config_names = []
for date in date_range:
    date = date.replace("-", "")
    for iters in [200, 100, 50]:
        mcts_log_name = f"{date}_{iters}IT_10CAP"
        config = dict()
        config["starting_date_str"] = date
        config["iter_limit"] = iters
        config["pool_thread_count"] = 10
        config["uct_tradeoff"] = 8074255
        config["mcts_discount_factor"] = 0.99997
        config["lookahead_horizon_delta_t"] = 3600
        config["rollout_horizon_delta_t"] = 3600
        config["allowed_computation_time"] = 15
        config["vehicle_count"] = "10CAP"
        config["oracle"] = False
        config["method"] = "MCTS"
        config["use_intervals"] = True
        config["use_timepoints"] = True
        config["send_mail"] = True
        config["save_metrics"] = False
        config["reallocation"] = True
        config["scenario"] = "1B"
        config["mcts_log_name"] = mcts_log_name
        
        with open(f'{configs_dir}/mcts_{date}_{iters}IT.json', 'w') as fp:
            json.dump(config, fp, indent=2)
            
        config_names.append(mcts_log_name)

devices = 2
windows = 2

date_range = list(np.array_split(date_range, devices))
for device in range(devices):
    lines = []
    d = list(np.array_split(date_range[device], windows))
    
    lines.append(f"session=device_{device}_200IT")
    lines.append("tmux has-session -t $session 2>/dev/null")
    lines.append("if [ $? != 0 ]; then")
    lines.append("  tmux new-session -d -s $session")
    lines.append("else")
    lines.append("  tmux kill-session -t $session")
    lines.append("fi")
    lines.append("")
    
    lines.append(f"window=0")
    # lines.append("tmux rename-window -t $session:$window '1'")
    
    for window in range(windows):
        lines.append("tmux send-keys 'conda activate py39' 'C-m'")
        lines.append("")
        
        for iter in [200]:
            for _d in d[window]:
                mcts_log_name = f"mcts_{_d.replace('-', '')}_{iter}IT"
                lines.append(f"cfile=\"configs/{mcts_log_name}\"")
                lines.append("logName=${cfile#*/}")
                lines.append("tmux send-keys 'python run_mcts_no_inject.py -c '$cfile'' C-m")
                lines.append("tmux send-keys 'python emailer.py -c '$cfile'' C-m")
                lines.append("tmux send-keys 'tmux capture-pane -pJ -S - > 'logs/$logName'.log' C-m")
                lines.append("")
        
        if window < windows - 1:
            lines.append("tmux split-window -h")
        lines.append("")

    lines.append("tmux select-layout tiled")
    lines.append("tmux a -t $session")
    
    with open(f'results/test_data/executers/execute_{device}_200IT.sh', 'w') as f:
        for line in lines:
            f.write(f"{line}\n")

In [ ]:
#### SAMPLE CONFIG.json ####

# {
#   "starting_date_str": "20210607",
#   "iter_limit": 200,
#   "pool_thread_count": 10,
#   "mcts_discount_factor": 0.99997,
#   "uct_tradeoff": 8074255,
#   "lookahead_horizon_delta_t": 3600,
#   "rollout_horizon_delta_t": 3600,
#   "allowed_computation_time": 15,
#   "vehicle_count": "",
#   "oracle": false,
#   "method": "MCTS",
#   "use_intervals": true,
#   "use_timepoints": true,
#   "save_metrics": false,
#   "send_mail": true,
#   "reallocation": true,
#   "early_end": false,
#   "scenario": "1B",
#   "mcts_log_name": "20210607_ALL_IT200_nobreak"
# }

In [ ]:
#### SAMPLE EXECUTOR.SH ####

# session="20210607"

# # Check if the session exists, discarding output
# # We can check $? for the exit status (zero for success, non-zero for failure)
# tmux has-session -t $session 2>/dev/null

# if [ $? != 0 ]; then
#   # Set up your session
#   tmux new-session -d -s $session
# else
#   tmux kill-session -t $session
# fi

# # ################################### WINDOW 0 ###################################

# window=0
# tmux rename-window -t $session:$window '1'
# tmux send-keys 'conda activate py39' 'C-m'
# cfile="configs/all_served"
# logName=${cfile#*/}
# tmux send-keys 'python run_mcts_no_inject.py -c '$cfile'' C-m
# tmux send-keys 'python emailer.py -c '$cfile'' C-m
# tmux send-keys 'tmux capture-pane -pJ -S - > 'logs/$logName'.log' C-m

# tmux split-window -h

# tmux send-keys 'conda activate py39' 'C-m'
# cfile="configs/all_served_100"
# logName=${cfile#*/}
# tmux send-keys 'python run_mcts_no_inject.py -c '$cfile'' C-m
# tmux send-keys 'python emailer.py -c '$cfile'' C-m
# tmux send-keys 'tmux capture-pane -pJ -S - > 'logs/$logName'.log' C-m
# tmux a -t $session
